In [1]:
import datetime
datetime.datetime.utcfromtimestamp(1713120612)

In [3]:
# manually remove last K dicts 
K = 6

with open('../../data/1pro_games.json', 'r', encoding='utf-8') as file:
        data = file.read()
        data = '[' + data[:-1] + ']'
        data = json.loads(data)

with open('../../data/1pro_games_AAA.json', 'w') as file:
    json.dump(data[:-K], file)

In [4]:
# CAST [{},{}] to {},{},
with open('../../data/1pro_games_new.json', 'r') as file:
    data = file.read()

data = data[1:]
data = data[:-1] + ','

with open('../../data/1pro_games_new.json', 'w') as file:
    file.write(data)


In [ ]:
# MERGE (without very last match, to keep tmp file non-empty)
import shutil
import json
import os

file_to_merge = r"D:\WORKSPACE\dotabet\data\1pro_games_2.json"
tmp_file = r"D:\WORKSPACE\dotabet\data\1pro_games_tmp.json"

directory = os.path.dirname(file_to_merge)
backup_file = os.path.join(directory, 'backup_' + os.path.basename(file_to_merge))
shutil.copyfile(file_to_merge, backup_file)

with open(file_to_merge, 'r', encoding='utf-8') as file:
        data1 = json.load(file)
    
mids1 = [m['match_id'] for m in data1]

with open(tmp_file, 'r', encoding='utf-8') as file:
        data2 = file.read()
        data2 = '[' + data2[:-1] + ']'
        data2 = json.loads(data2)

for match2 in data2:
    if match2['match_id'] in mids1:
        print(f"❌ Match {match2['match_id']} from {tmp_file} is already present in {file_to_merge}")
    
with open(file_to_merge, 'w') as file:
    json.dump(data1+data2[:-1], file, separators=(',', ':'))

json_string = json.dumps(data2[-1], ensure_ascii=False, separators=(',', ': '), default=lambda x: None if x is None else str(x).lower())
with open(tmp_file, 'w') as file:
    file.write(json_string+',')

In [8]:
# CHECK UNIQUE
file_path = r"D:\WORKSPACE\dotabet\data\1pro_games_2.json"

with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    
mids = [m['match_id'] for m in data]
len(mids), len(set(mids))

(4489, 4489)

In [ ]:
4413, 4413)

In [7]:
# FILTER (REMOVE DUPLICATS)
import json
file_to_modify = r"D:\WORKSPACE\dotabet\data\1pro_games_2.json"

with open(file_to_modify, 'r', encoding='utf-8') as file:
        data = json.load(file)

mids = []
modified_data = []
for d in data:
    if d.get('match_id') not in mids:
        modified_data.append(d)
        mids.append(d.get('match_id'))
    else:
        print(f"Find duplicat {d.get('match_id')}")
    
with open(file_to_modify, 'w') as file:
    json.dump(modified_data, file, separators=(',', ':'))

Find duplicat 7686486497
Find duplicat 7717209834
Find duplicat 7717196941
Find duplicat 7717120872
Find duplicat 7717103823
Find duplicat 7717099626
Find duplicat 7717035183
Find duplicat 7716979372
Find duplicat 7716934124
Find duplicat 7716909071
Find duplicat 7716869143
Find duplicat 7716861141
Find duplicat 7716813674
Find duplicat 7716806803
Find duplicat 7716796011
Find duplicat 7716724170
Find duplicat 7716697572
Find duplicat 7716693953
Find duplicat 7716665819
Find duplicat 7716665738
Find duplicat 7716663098
Find duplicat 7716653833
Find duplicat 7716650838
Find duplicat 7716635117
Find duplicat 7716634082
Find duplicat 7716603570
Find duplicat 7716598778
Find duplicat 7716595869
Find duplicat 7716593877
Find duplicat 7716592900
Find duplicat 7716557334
Find duplicat 7716552395
Find duplicat 7716546710
Find duplicat 7716519574
Find duplicat 7716508168
Find duplicat 7716501701
Find duplicat 7716501633
Find duplicat 7716493932
Find duplicat 7716491024
Find duplicat 7716472947
